In [1]:

import multiprocessing
import os
import tempfile
import time
from multiprocessing import Pool
from typing import Callable, Dict, Optional, Tuple

import fsspec
import pandas as pd
import requests
from loguru import logger
from tqdm import tqdm

from objaverse.utils import get_file_hash
from objaverse.xl.abstract import ObjaverseSource



In [2]:
import objaverse.xl as oxl
annotations = oxl.get_annotations(
    download_dir="~/.objaverse" # default download directory
)
annotations

2024-02-01 19:10:36.885 | INFO     | objaverse.xl.github:_get_annotations:65 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/github/github.parquet to ~/.objaverse/github/github.parquet
2024-02-01 19:10:57.198 | INFO     | objaverse.xl.thingiverse:get_annotations:46 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/thingiverse/thingiverse.parquet to ~/.objaverse/thingiverse/thingiverse.parquet
2024-02-01 19:11:06.360 | INFO     | objaverse.xl.smithsonian:get_annotations:47 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/smithsonian/smithsonian.parquet to ~/.objaverse/smithsonian/smithsonian.parquet
2024-02-01 19:11:06.561 | INFO     | objaverse.xl.sketchfab:_get_annotations:52 - Downloading https://huggingface.co/datasets/allenai/objaverse-xl/resolve/main/sketchfab/sketchfab.parquet to ~/.objaverse/sketchfab/sketchfab.parquet


,fileIdentifier,source,license,fileType,sha256,metadata
0,https://github.com/GameGC/Testovoe-SPB-Git/blo...,github,None,obj,fca3990d6d91e110cb920aa3da1a84e54b4a00cde7ad1e...,{}
1,https://github.com/9-zzz/RitualRex-GGJ2016/blo...,github,None,blend,09dc2689b8d0510885e19a7b525ee44709849d2601add8...,{}
2,https://github.com/mattoverby/mesh-data/blob/2...,github,MIT License,obj,f9d17190b54b548c6a3e6feebe7b2951a9b8060c507874...,{}
3,https://github.com/0010cha/data_origami/blob/0...,github,MIT License,ply,f25184898312af8ed5e77cfe675d2251c9893ac5ccc3a1...,{}
4,https://github.com/Xrvitd/Point2Skeleton_withC...,github,MIT License,ply,d721e6b1b97a83d1f6700045efcb478a1db4686b0571f6...,{}
...,...,...,...,...,...,...
9767006,https://sketchfab.com/3d-models/cc2c9ac5149b40...,sketchfab,Creative Commons - Attribution,glb,76cc3e0343be2452fb7a1d8f6eb295f9067e859f353bb8...,{}
9767007,https://sketchfab.com/3d-models/48ad63b00c4448...,sketchfab,Creative Commons - Attribution,glb,15524e631cecaf3537bbbd9756a94ffa78b275ce77e683...,{}
9767008,https://sketchfab.com/3d-models/3f61c303139f4d...,sketchfab,Creative Commons - Attribution,glb,4b4a2c0e5f4bd89b15c9551c7c34c0083774ead2fb6354...,{}
9767009,https://sketchfab.com/3d-models/bc79ba06d75740...,sketchfab,Creative Commons - Attribution,glb,9e71bc509985344446e075bac884cd1a76c24b09f82217...,{}


In [3]:
from typing import Any, Dict, Hashable

def handle_found_object(
    local_path: str,
    file_identifier: str,
    sha256: str,
    metadata: Dict[Hashable, Any]
) -> None:
    print("\n\n\n---HANDLE_FOUND_OBJECT CALLED---\n",
          f"  {local_path=}\n  {file_identifier=}\n  {sha256=}\n  {metadata=}\n\n\n")

In [4]:
def handle_modified_object(
    local_path: str,
    file_identifier: str,
    new_sha256: str,
    old_sha256: str,
    metadata: Dict[Hashable, Any],
) -> None:
    print("\n\n\n---HANDLE_MODIFIED_OBJECT CALLED---\n",
          f"  {local_path=}\n  {file_identifier=}\n  {old_sha256=}\n  {new_sha256}\n  {metadata=}\n\n\n")

In [5]:
def handle_missing_object(
    file_identifier: str,
    sha256: str,
    metadata: Dict[Hashable, Any]
) -> None:
    print("\n\n\n---HANDLE_MISSING_OBJECT CALLED---\n",
          f"  {file_identifier=}\n  {sha256=}\n  {metadata=}\n\n\n")

In [6]:
# oxl.download_objects(
#     # Base parameters:
#     objects = annotations,
#     download_dir = "~/objaverse",
#     processes = 18,  # None => multiprocessing.cpu_count()
# )

2024-02-01 19:16:21.611 | INFO     | objaverse.xl.smithsonian:download_objects:312 - Found 0 Smithsonian Objects already downloaded
2024-02-01 19:16:21.612 | INFO     | objaverse.xl.smithsonian:download_objects:315 - Downloading 2407 Smithsonian Objects with 18 processes
2024-02-01 19:18:07.037 | WARNING  | objaverse.xl.smithsonian:_download_smithsonian_object:154 - 404 for https://3d-api.si.edu/content/document/3d_package:d8c64e02-4ebc-11ea-b77f-2e728ce88125/chandra_fek_master-150k-2048-medium.glb
2024-02-01 19:18:07.047 | WARNING  | objaverse.xl.smithsonian:_download_smithsonian_object:154 - 404 for https://3d-api.si.edu/content/document/3d_package:d8c64e02-4ebc-11ea-b77f-2e728ce88125/fiducial_jets_master-150k-2048-medium.glb
2024-02-01 19:18:07.098 | WARNING  | objaverse.xl.smithsonian:_download_smithsonian_object:154 - 404 for https://3d-api.si.edu/content/document/3d_package:d8c64e02-4ebc-11ea-b77f-2e728ce88125/neutron_star_master-150k-2048-medium.glb
2024-02-01 19:18:07.178 | WAR

KeyboardInterrupt: 